In [1]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing 
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.wrappers.scikit_learn import KerasRegressor

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import tensorflow as tf

from sklearn.feature_selection import VarianceThreshold

Using TensorFlow backend.


In [3]:
data_p = pd.read_csv("drive/My Drive/divija/points.csv",dtype=object,error_bad_lines=False) 
data_p.head()
data_p["id"] = data_p["id"].map(str) +"_"+ data_p["dir"]
data_p.head()
# data_p.dtypes

,s.no,id,dir,path,date,x1,y1,x2,y2,x3,y3,x4,y4,x5,y5,x6,y6,x7,y7,x8,y8,x9,y9,x10,y10,x11,y11,x12,y12,x13,y13,x14,y14,x15,y15,x16,y16,x17,y17,x18,...,x9981,y9981,x9982,y9982,x9983,y9983,x9984,y9984,x9985,y9985,x9986,y9986,x9987,y9987,x9988,y9988,x9989,y9989,x9990,y9990,x9991,y9991,x9992,y9992,x9993,y9993,x9994,y9994,x9995,y9995,x9996,y9996,x9997,y9997,x9998,y9998,x9999,y9999,x10000,y10000
0,1,ON_1_20181031_173504_104,104,C:\Hilink\BB4A\RESULT\BBM2\BBM2\NOMINAL\MANUAL...,31/10/2018,0.2,0.03,0.2,0.03,0.2,0.03,0.2,0.03,0.2,0.03,0.0,0.03,0.2,0.03,0.2,0.03,0.4,0.03,0.0,0.03,0.0,0.03,0.2,0.03,0.4,0.03,0.0,0.03,0.2,0.03,0.2,0.03,0.0,0.03,0.2,...,0.0,0.15,0.4,0.15,0.2,0.14,0.2,0.15,0.2,0.14,0.0,0.14,0.2,0.15,0.2,0.15,0.2,0.14,0.2,0.14,0.4,0.15,0.2,0.14,0.2,0.14,0.2,0.14,0.2,0.15,0.2,0.14,0.2,0.14,0.2,0.14,0.2,0.14,0.2,0.14
1,2,OFF_4_20181031_173921_104,104,C:\Hilink\BB4A\RESULT\BBM2\BBM2\NOMINAL\MANUAL...,31/10/2018,0.4,0.12,0.2,0.12,0.2,0.12,0.2,0.12,0.0,0.12,0.4,0.12,0.2,0.12,0.4,0.12,0.2,0.12,0.2,0.12,0.0,0.12,0.2,0.11,0.2,0.12,0.2,0.12,0.2,0.12,0.2,0.12,0.2,0.12,0.2,...,25.0,0.03,25.0,0.03,25.4,0.03,25.0,0.03,25.2,0.03,25.0,0.03,25.0,0.03,25.0,0.03,25.2,0.03,25.2,0.03,25.0,0.03,25.2,0.03,25.2,0.03,25.0,0.03,25.0,0.03,25.0,0.03,25.0,0.03,25.0,0.03,25.0,0.03,25.0,0.03
2,3,OFF_2_20181031_173628_104,104,C:\Hilink\BB4A\RESULT\BBM2\BBM2\NOMINAL\MANUAL...,31/10/2018,0.2,0.13,0.0,0.12,0.0,0.12,0.0,0.12,0.0,0.12,0.2,0.12,0.2,0.12,0.2,0.12,0.4,0.12,0.2,0.13,0.2,0.13,0.2,0.12,0.2,0.12,0.2,0.13,0.2,0.12,0.2,0.13,0.0,0.12,0.2,...,25.0,0.03,25.2,0.03,25.2,0.02,25.2,0.03,25.4,0.03,25.0,0.03,25.0,0.03,25.0,0.03,25.0,0.03,25.2,0.03,25.0,0.03,25.0,0.03,25.0,0.03,25.0,0.03,25.0,0.03,25.0,0.03,25.0,0.03,25.0,0.02,25.0,0.02,25.0,0.03
3,4,ON_3_20181031_173800_104,104,C:\Hilink\BB4A\RESULT\BBM2\BBM2\NOMINAL\MANUAL...,31/10/2018,0.2,0.03,0.4,0.02,0.4,0.04,0.2,0.03,0.4,0.03,0.2,0.03,0.2,0.04,0.0,0.03,0.2,0.03,0.2,0.03,0.4,0.03,0.4,0.03,0.2,0.03,0.2,0.03,0.4,0.03,0.0,0.03,0.2,0.03,0.2,...,0.2,0.14,0.2,0.14,0.0,0.14,0.0,0.14,0.2,0.14,0.2,0.14,0.4,0.14,0.4,0.14,0.2,0.14,0.2,0.14,0.2,0.14,0.2,0.15,0.2,0.14,0.2,0.14,0.2,0.15,0.2,0.14,0.0,0.14,0.2,0.14,0.2,0.14,0.2,0.14
4,5,OFF_4_20181102_085018_103,103,C:\Hilink\BB4A\RESULT\BBM2\BBM2\NOMINAL\MANUAL...,02/11/2018,0.2,0.15,0.2,0.16,0.2,0.15,0.2,0.15,0.0,0.15,0.2,0.15,0.0,0.15,0.2,0.15,0.2,0.14,0.0,0.15,0.2,0.15,0.2,0.14,0.2,0.15,0.2,0.15,0.0,0.15,0.2,0.15,0.2,0.15,0.2,...,25.0,0.05,25.2,0.06,25.2,0.05,25.0,0.06,25.2,0.06,25.0,0.05,25.0,0.06,25.0,0.06,25.0,0.06,25.0,0.06,25.2,0.06,25.0,0.06,24.8,0.06,25.0,0.06,25.0,0.06,25.0,0.05,25.0,0.06,25.0,0.05,25.2,0.05,25.0,0.05


In [4]:

data_v = pd.read_csv("drive/My Drive/divija/values.csv",dtype=object,error_bad_lines=False )
le = preprocessing.LabelEncoder()

data_v['power_state_spec'] = le.fit_transform(data_v['power_state_spec'].astype('str'))

data_v['power_state_value'] = le.fit_transform(data_v['power_state_value'].astype('str'))
data_v["id"] = data_v["id"].map(str) +"_"+data_v["dir"]
data_v.head()

,s.no,id,dir,_file_,power_state_value,current_rise/fall_time_value (mS),current_stabilised_value (mA),current_max/min_value (mA),power_state_spec,current_rise/fall_time_spec (mS),current_stabilised_spec (mA),current_max/min_spec (mA),power_state_N/NC,current_rise/fall_time_C/NC,current_stabilised_C/NC,current_max/min_C/NC
0,1,ON_1_20181031_173504_104,104,HEGSE_72.HTM,1,89.990000000000,150.000000000000,405.992200000000,1,60,510.204,800,C,NC,C,C
1,2,OFF_2_20181031_173628_104,104,HEGSE_72.HTM,0,7.992000000000,30.000000000000,-56.000000000000,0,10,510.204,-100,C,C,C,C
2,3,ON_3_20181031_173800_104,104,HEGSE_72.HTM,1,89.950000000000,140.000000000000,405.992200000000,1,60,495.049,800,C,NC,C,C
3,4,OFF_4_20181031_173921_104,104,HEGSE_72.HTM,0,7.997000000000,30.000000000000,-58.000000000000,0,10,495.049,-100,C,C,C,C
4,5,ON_1_20181102_084600_103,103,HEGSE_72.HTM,1,56.650000000000,169.800000000000,405.992200000000,1,60,510.204,800,C,C,C,C


In [0]:
arr_v = data_v.to_numpy()
arr_p = data_p.to_numpy()

In [6]:
arr_v = arr_v[0:]
print(arr_v)
arr_p = arr_p[0:]
print(arr_p)

[['1' 'ON_1_20181031_173504_104' '104' ... 'NC' 'C' 'C']
 ['2' 'OFF_2_20181031_173628_104' '104' ... 'C' 'C' 'C']
 ['3' 'ON_3_20181031_173800_104' '104' ... 'NC' 'C' 'C']
 ...
 ['767' 'OFF_2_20181113_232240_113' '113' ... 'C' 'NC' 'C']
 ['768' 'ON_3_20181113_232417_113' '113' ... 'C' 'C' 'C']
 ['769' 'OFF_4_20181113_232541_113' '113' ... 'C' 'NC' 'C']]
[['1' 'ON_1_20181031_173504_104' '104' ... '0.14' '0.2' '0.14']
 ['2' 'OFF_4_20181031_173921_104' '104' ... '0.03' '25.0' '0.03']
 ['3' 'OFF_2_20181031_173628_104' '104' ... '0.02' '25.0' '0.03']
 ...
 ['822' 'OFF_2_20181113_232240_113' '113' ... '0.05' '24.9359375' '0.05']
 ['823' 'OFF_4_20181113_232541_113' '113' ... '0.05' '24.9046875' '0.05']
 ['824' 'ON_3_20181113_232417_113' '113' ... '0.16' '0.278125' '0.16']]


In [7]:
ON_list =[]
OFF_list = []
for i in range(len(arr_p)):
    s = arr_p[i][1]
    s = str(s)
    
#     print(type(st))
    if s.find("N") == -1:
        OFF_list.append(arr_p[i])
    
    else:
        ON_list.append(arr_p[i])
# calculating for ON
print(len(ON_list),"ON")
print(len(OFF_list),"OFF")
arr_on_p = np.array(ON_list)
print(arr_on_p)

426 ON
398 OFF
[['1' 'ON_1_20181031_173504_104' '104' ... '0.14' '0.2' '0.14']
 ['4' 'ON_3_20181031_173800_104' '104' ... '0.14' '0.2' '0.14']
 ['7' 'ON_3_20181102_084856_103' '103' ... '0.17' '0.2' '0.17']
 ...
 ['819' 'ON_3_20181117_171650_114' '114' ... '0.15' '0.32109375' '0.15']
 ['821' 'ON_1_20181113_232117_113' '113' ... '0.17' '0.284375' '0.17']
 ['824' 'ON_3_20181113_232417_113' '113' ... '0.16' '0.278125' '0.16']]


In [8]:
arr_on_p = np.delete(arr_on_p, 3,  axis=1)
arr_on_p_n = arr_on_p[:, 1::2]
arr_on_p_f = np.delete(arr_on_p_n, 1,  axis=1)
print(len(arr_on_p_f))
print(len(arr_on_p_f[0]))
print(arr_on_p_f[0])

426
10001
['ON_1_20181031_173504_104' '0.03' '0.03' ... '0.14' '0.14' '0.14']


In [0]:

arr_on_p_f[:,1:]= arr_on_p_f[:,1:].astype('float64')
# print((arr_on_p_f[413]))

In [10]:
data = arr_on_p_f

df=pd.DataFrame(data=data[0:,0:],index=[i for i in range(data.shape[0])],
                columns=['y'+str(i) for i in range(data.shape[1])])
df.head()
# df.dtypes


,y0,y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11,y12,y13,y14,y15,y16,y17,y18,y19,y20,y21,y22,y23,y24,y25,y26,y27,y28,y29,y30,y31,y32,y33,y34,y35,y36,y37,y38,y39,...,y9961,y9962,y9963,y9964,y9965,y9966,y9967,y9968,y9969,y9970,y9971,y9972,y9973,y9974,y9975,y9976,y9977,y9978,y9979,y9980,y9981,y9982,y9983,y9984,y9985,y9986,y9987,y9988,y9989,y9990,y9991,y9992,y9993,y9994,y9995,y9996,y9997,y9998,y9999,y10000
0,ON_1_20181031_173504_104,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.14,0.14,0.14,0.14,0.15,0.14,0.14,0.15,0.15,0.14,0.15,0.14,0.14,0.15,0.14,0.14,0.15,0.15,0.14,0.14,0.15,0.15,0.14,0.15,0.14,0.14,0.15,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14
1,ON_3_20181031_173800_104,0.03,0.02,0.04,0.03,0.03,0.03,0.04,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.04,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14
2,ON_3_20181102_084856_103,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.05,0.05,0.06,0.05,0.06,0.05,0.05,0.06,0.06,0.06,0.05,0.06,0.05,0.06,0.05,0.06,0.05,0.05,0.06,0.06,0.06,0.06,0.05,0.05,0.05,...,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.16,0.16,0.17,0.17,0.17,0.16,0.16,0.16,0.17,0.16,0.17,0.16,0.17,0.16,0.16,0.17,0.16,0.16,0.16,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.17,0.17,0.17
3,ON_1_20181102_084600_103,0.05,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,...,0.17,0.16,0.17,0.17,0.17,0.18,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.16,0.16,0.17,0.17,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.16,0.17,0.17,0.16,0.16,0.17,0.17,0.17,0.17,0.17,0.16,0.17
4,ON_1_20181102_085847_102,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,...,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17


In [11]:
print(df.shape)
# for j in range(10000):
#   var = "y"+str(j+1)
#   df[var].fillna(df[var].mean(), inplace=True)
df_no_miss = df.dropna()
print(df_no_miss.shape)
print(df.shape)
df_no_miss.head()

(426, 10001)
(423, 10001)
(426, 10001)


,y0,y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11,y12,y13,y14,y15,y16,y17,y18,y19,y20,y21,y22,y23,y24,y25,y26,y27,y28,y29,y30,y31,y32,y33,y34,y35,y36,y37,y38,y39,...,y9961,y9962,y9963,y9964,y9965,y9966,y9967,y9968,y9969,y9970,y9971,y9972,y9973,y9974,y9975,y9976,y9977,y9978,y9979,y9980,y9981,y9982,y9983,y9984,y9985,y9986,y9987,y9988,y9989,y9990,y9991,y9992,y9993,y9994,y9995,y9996,y9997,y9998,y9999,y10000
0,ON_1_20181031_173504_104,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.14,0.14,0.14,0.14,0.15,0.14,0.14,0.15,0.15,0.14,0.15,0.14,0.14,0.15,0.14,0.14,0.15,0.15,0.14,0.14,0.15,0.15,0.14,0.15,0.14,0.14,0.15,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14
1,ON_3_20181031_173800_104,0.03,0.02,0.04,0.03,0.03,0.03,0.04,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.04,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14
2,ON_3_20181102_084856_103,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.05,0.05,0.06,0.05,0.06,0.05,0.05,0.06,0.06,0.06,0.05,0.06,0.05,0.06,0.05,0.06,0.05,0.05,0.06,0.06,0.06,0.06,0.05,0.05,0.05,...,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.16,0.16,0.17,0.17,0.17,0.16,0.16,0.16,0.17,0.16,0.17,0.16,0.17,0.16,0.16,0.17,0.16,0.16,0.16,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.17,0.17,0.17
3,ON_1_20181102_084600_103,0.05,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,...,0.17,0.16,0.17,0.17,0.17,0.18,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.16,0.16,0.17,0.17,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.16,0.17,0.17,0.16,0.16,0.17,0.17,0.17,0.17,0.17,0.16,0.17
4,ON_1_20181102_085847_102,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,...,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17


In [12]:
arr_p_no = df_no_miss.to_numpy()
print(len(arr_p_no))

423


In [13]:
df1= df_no_miss.rename(index=str, columns={"y0": "id"})
df1.head()

,id,y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11,y12,y13,y14,y15,y16,y17,y18,y19,y20,y21,y22,y23,y24,y25,y26,y27,y28,y29,y30,y31,y32,y33,y34,y35,y36,y37,y38,y39,...,y9961,y9962,y9963,y9964,y9965,y9966,y9967,y9968,y9969,y9970,y9971,y9972,y9973,y9974,y9975,y9976,y9977,y9978,y9979,y9980,y9981,y9982,y9983,y9984,y9985,y9986,y9987,y9988,y9989,y9990,y9991,y9992,y9993,y9994,y9995,y9996,y9997,y9998,y9999,y10000
0,ON_1_20181031_173504_104,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.14,0.14,0.14,0.14,0.15,0.14,0.14,0.15,0.15,0.14,0.15,0.14,0.14,0.15,0.14,0.14,0.15,0.15,0.14,0.14,0.15,0.15,0.14,0.15,0.14,0.14,0.15,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14
1,ON_3_20181031_173800_104,0.03,0.02,0.04,0.03,0.03,0.03,0.04,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.04,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14
2,ON_3_20181102_084856_103,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.05,0.05,0.06,0.05,0.06,0.05,0.05,0.06,0.06,0.06,0.05,0.06,0.05,0.06,0.05,0.06,0.05,0.05,0.06,0.06,0.06,0.06,0.05,0.05,0.05,...,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.16,0.16,0.17,0.17,0.17,0.16,0.16,0.16,0.17,0.16,0.17,0.16,0.17,0.16,0.16,0.17,0.16,0.16,0.16,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.17,0.17,0.17
3,ON_1_20181102_084600_103,0.05,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,...,0.17,0.16,0.17,0.17,0.17,0.18,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.16,0.16,0.17,0.17,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.16,0.17,0.17,0.16,0.16,0.17,0.17,0.17,0.17,0.17,0.16,0.17
4,ON_1_20181102_085847_102,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,...,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17


In [14]:
print(df1.shape)
df2 = data_v
print(df2.shape)
# df2['revenue'] = df2.CET.map(df1.set_index('date')['revenue'])

# to find out number of times a id is repeating
# print(df1.id.value_counts())


# print(df1['id'].nunique())
# print(df2['id'].nunique())

# print(df1.duplicated(['id','y1','y7897']).sum())

# print(df2.duplicated(['id']).sum())
combine = (pd.merge(df1, df2, how='left', on='id'))
# print(df1.unique)
print(combine.shape)

(423, 10001)
(769, 16)
(423, 10016)


In [15]:
combine.head()

,id,y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11,y12,y13,y14,y15,y16,y17,y18,y19,y20,y21,y22,y23,y24,y25,y26,y27,y28,y29,y30,y31,y32,y33,y34,y35,y36,y37,y38,y39,...,y9976,y9977,y9978,y9979,y9980,y9981,y9982,y9983,y9984,y9985,y9986,y9987,y9988,y9989,y9990,y9991,y9992,y9993,y9994,y9995,y9996,y9997,y9998,y9999,y10000,s.no,dir,_file_,power_state_value,current_rise/fall_time_value (mS),current_stabilised_value (mA),current_max/min_value (mA),power_state_spec,current_rise/fall_time_spec (mS),current_stabilised_spec (mA),current_max/min_spec (mA),power_state_N/NC,current_rise/fall_time_C/NC,current_stabilised_C/NC,current_max/min_C/NC
0,ON_1_20181031_173504_104,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.14,0.15,0.15,0.14,0.14,0.15,0.15,0.14,0.15,0.14,0.14,0.15,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14,1,104,HEGSE_72.HTM,1.0,89.990000000000,150.000000000000,405.992200000000,1.0,60,510.204,800,C,NC,C,C
1,ON_3_20181031_173800_104,0.03,0.02,0.04,0.03,0.03,0.03,0.04,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.04,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.14,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14,3,104,HEGSE_72.HTM,1.0,89.950000000000,140.000000000000,405.992200000000,1.0,60,495.049,800,C,NC,C,C
2,ON_3_20181102_084856_103,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.05,0.05,0.06,0.05,0.06,0.05,0.05,0.06,0.06,0.06,0.05,0.06,0.05,0.06,0.05,0.06,0.05,0.05,0.06,0.06,0.06,0.06,0.05,0.05,0.05,...,0.17,0.16,0.17,0.16,0.17,0.16,0.16,0.17,0.16,0.16,0.16,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.17,0.17,0.17,7,103,HEGSE_72.HTM,1.0,36.970000000000,166.300000000000,405.992200000000,1.0,60,495.049,800,C,C,C,C
3,ON_1_20181102_084600_103,0.05,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,...,0.17,0.17,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.16,0.17,0.17,0.16,0.16,0.17,0.17,0.17,0.17,0.17,0.16,0.17,5,103,HEGSE_72.HTM,1.0,56.650000000000,169.800000000000,405.992200000000,1.0,60,510.204,800,C,C,C,C
4,ON_1_20181102_085847_102,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,...,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,9,102,HEGSE_72.HTM,1.0,37.040000000000,170.000000000000,401.312500000000,1.0,60,510.204,800,C,C,C,C


In [16]:
combine.iloc[:,0:10010].head()
k = combine.drop(['s.no','dir','_file_'], axis = 1) 

k.head()

,id,y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11,y12,y13,y14,y15,y16,y17,y18,y19,y20,y21,y22,y23,y24,y25,y26,y27,y28,y29,y30,y31,y32,y33,y34,y35,y36,y37,y38,y39,...,y9973,y9974,y9975,y9976,y9977,y9978,y9979,y9980,y9981,y9982,y9983,y9984,y9985,y9986,y9987,y9988,y9989,y9990,y9991,y9992,y9993,y9994,y9995,y9996,y9997,y9998,y9999,y10000,power_state_value,current_rise/fall_time_value (mS),current_stabilised_value (mA),current_max/min_value (mA),power_state_spec,current_rise/fall_time_spec (mS),current_stabilised_spec (mA),current_max/min_spec (mA),power_state_N/NC,current_rise/fall_time_C/NC,current_stabilised_C/NC,current_max/min_C/NC
0,ON_1_20181031_173504_104,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.14,0.15,0.14,0.14,0.15,0.15,0.14,0.14,0.15,0.15,0.14,0.15,0.14,0.14,0.15,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14,1.0,89.990000000000,150.000000000000,405.992200000000,1.0,60,510.204,800,C,NC,C,C
1,ON_3_20181031_173800_104,0.03,0.02,0.04,0.03,0.03,0.03,0.04,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.04,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.14,0.14,0.14,0.14,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14,1.0,89.950000000000,140.000000000000,405.992200000000,1.0,60,495.049,800,C,NC,C,C
2,ON_3_20181102_084856_103,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.05,0.05,0.06,0.05,0.06,0.05,0.05,0.06,0.06,0.06,0.05,0.06,0.05,0.06,0.05,0.06,0.05,0.05,0.06,0.06,0.06,0.06,0.05,0.05,0.05,...,0.16,0.16,0.16,0.17,0.16,0.17,0.16,0.17,0.16,0.16,0.17,0.16,0.16,0.16,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.17,0.17,0.17,1.0,36.970000000000,166.300000000000,405.992200000000,1.0,60,495.049,800,C,C,C,C
3,ON_1_20181102_084600_103,0.05,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,...,0.17,0.16,0.16,0.17,0.17,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.16,0.17,0.17,0.16,0.16,0.17,0.17,0.17,0.17,0.17,0.16,0.17,1.0,56.650000000000,169.800000000000,405.992200000000,1.0,60,510.204,800,C,C,C,C
4,ON_1_20181102_085847_102,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,...,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,1.0,37.040000000000,170.000000000000,401.312500000000,1.0,60,510.204,800,C,C,C,C


In [17]:
input_1 = k.iloc[:,0:10009]
# filling the missing values

# print(input_1.iloc[:,10008])
miss = input_1.iloc[:,1:]
miss.head()

,y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11,y12,y13,y14,y15,y16,y17,y18,y19,y20,y21,y22,y23,y24,y25,y26,y27,y28,y29,y30,y31,y32,y33,y34,y35,y36,y37,y38,y39,y40,...,y9969,y9970,y9971,y9972,y9973,y9974,y9975,y9976,y9977,y9978,y9979,y9980,y9981,y9982,y9983,y9984,y9985,y9986,y9987,y9988,y9989,y9990,y9991,y9992,y9993,y9994,y9995,y9996,y9997,y9998,y9999,y10000,power_state_value,current_rise/fall_time_value (mS),current_stabilised_value (mA),current_max/min_value (mA),power_state_spec,current_rise/fall_time_spec (mS),current_stabilised_spec (mA),current_max/min_spec (mA)
0,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.15,0.14,0.15,0.14,0.14,0.15,0.14,0.14,0.15,0.15,0.14,0.14,0.15,0.15,0.14,0.15,0.14,0.14,0.15,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14,1.0,89.990000000000,150.000000000000,405.992200000000,1.0,60,510.204,800
1,0.03,0.02,0.04,0.03,0.03,0.03,0.04,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.04,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14,1.0,89.950000000000,140.000000000000,405.992200000000,1.0,60,495.049,800
2,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.05,0.05,0.06,0.05,0.06,0.05,0.05,0.06,0.06,0.06,0.05,0.06,0.05,0.06,0.05,0.06,0.05,0.05,0.06,0.06,0.06,0.06,0.05,0.05,0.05,0.06,...,0.16,0.17,0.17,0.17,0.16,0.16,0.16,0.17,0.16,0.17,0.16,0.17,0.16,0.16,0.17,0.16,0.16,0.16,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.17,0.17,0.17,1.0,36.970000000000,166.300000000000,405.992200000000,1.0,60,495.049,800
3,0.05,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,...,0.17,0.17,0.17,0.17,0.17,0.16,0.16,0.17,0.17,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.16,0.17,0.17,0.16,0.16,0.17,0.17,0.17,0.17,0.17,0.16,0.17,1.0,56.650000000000,169.800000000000,405.992200000000,1.0,60,510.204,800
4,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,...,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,1.0,37.040000000000,170.000000000000,401.312500000000,1.0,60,510.204,800


In [18]:
input_1 = k.iloc[:,0:10009]
# filling the missing values

# print(input_1.iloc[:,10008])
miss = input_1.iloc[:,1:]
miss.head()


# miss.columns[miss.isna().any()].tolist()
# for j in range((miss.shape[0])):
#   if miss["current_rise/fall_time_value (mS)"].notnull() == True:
#     print(j)
  
for column in (miss.iloc[:,10000:]):
  su = 0
  div = 0
  for r in range(miss.shape[0]):
    if (pd.isna(miss[column][r]))== False:
      su = float(miss[column][r])+su

      div = div+1

  fin = float(su/div)

  miss[column].fillna(float(fin),inplace=True)
#
 ################TO CHECK IS THERE IS ANY NAN ############################## 
#   if miss[column].isnull().any():

    
#     print('{0} has {1} null values'.format('current_rise/fall_time_value (mS)', miss['current_rise/fall_time_value (mS)'].isnull().sum()))
#   else:
#     print("no")
    
    


# if(pd.isna(miss["current_rise/fall_time_value (mS)"][23]))== True:


# print(miss.dtypes)
#########converting ever
# thing into float
miss =miss.astype('float64')
# print(miss.dtypes)
miss.head()


,y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11,y12,y13,y14,y15,y16,y17,y18,y19,y20,y21,y22,y23,y24,y25,y26,y27,y28,y29,y30,y31,y32,y33,y34,y35,y36,y37,y38,y39,y40,...,y9969,y9970,y9971,y9972,y9973,y9974,y9975,y9976,y9977,y9978,y9979,y9980,y9981,y9982,y9983,y9984,y9985,y9986,y9987,y9988,y9989,y9990,y9991,y9992,y9993,y9994,y9995,y9996,y9997,y9998,y9999,y10000,power_state_value,current_rise/fall_time_value (mS),current_stabilised_value (mA),current_max/min_value (mA),power_state_spec,current_rise/fall_time_spec (mS),current_stabilised_spec (mA),current_max/min_spec (mA)
0,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.15,0.14,0.15,0.14,0.14,0.15,0.14,0.14,0.15,0.15,0.14,0.14,0.15,0.15,0.14,0.15,0.14,0.14,0.15,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14,1.0,89.99,150.0,405.9922,1.0,60.0,510.204,800.0
1,0.03,0.02,0.04,0.03,0.03,0.03,0.04,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.04,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14,1.0,89.95,140.0,405.9922,1.0,60.0,495.049,800.0
2,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.05,0.05,0.06,0.05,0.06,0.05,0.05,0.06,0.06,0.06,0.05,0.06,0.05,0.06,0.05,0.06,0.05,0.05,0.06,0.06,0.06,0.06,0.05,0.05,0.05,0.06,...,0.16,0.17,0.17,0.17,0.16,0.16,0.16,0.17,0.16,0.17,0.16,0.17,0.16,0.16,0.17,0.16,0.16,0.16,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.17,0.17,0.17,1.0,36.97,166.3,405.9922,1.0,60.0,495.049,800.0
3,0.05,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,...,0.17,0.17,0.17,0.17,0.17,0.16,0.16,0.17,0.17,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.16,0.17,0.17,0.16,0.16,0.17,0.17,0.17,0.17,0.17,0.16,0.17,1.0,56.65,169.8,405.9922,1.0,60.0,510.204,800.0
4,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,...,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,1.0,37.04,170.0,401.3125,1.0,60.0,510.204,800.0


In [0]:
import pandas as pd
from sklearn.ensemble.forest import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import robust_scale

scaler_min_x = MinMaxScaler()
scaler_min_y = MinMaxScaler()

scaler_norm_x = Normalizer()
scaler_norm_y = Normalizer()

scaler_stan_x = StandardScaler()
scaler_stan_y = StandardScaler()

scalar_qt_x =QuantileTransformer(output_distribution='uniform')
scalar_qt_y =QuantileTransformer(output_distribution='uniform')
       

In [20]:
rand_na = miss
# print(miss.shape)
input_1_arr = rand_na.to_numpy()
input_1_arr[:,:]= input_1_arr[:,:].astype('float64')

X = input_1_arr[:,0:10000]
Y = input_1_arr[:,10002:10004]
# print(X.shape)
# print(Y.shape)
# print(Y)
y=np.reshape(Y, (-1,1))

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20)

# ######minmax
scaler_min_x = MinMaxScaler().fit(X_train)
scaler_min_y = MinMaxScaler().fit(y_train)

X_minmax_train = scaler_min_x.transform(X_train)
Y_minmax_train = scaler_min_y.transform(y_train)


# print(X)
# print(Y)
#####standard

scaler_stan_x = StandardScaler().fit(X_train)
scaler_stan_y = StandardScaler().fit(y_train)


X_stan_train = scaler_stan_x.transform(X_train)
Y_stan_train = scaler_stan_y.transform(y_train)

#######normlised
scaler_norm_x = Normalizer().fit(X_train)
scaler_norm_y = Normalizer().fit(y_train)


X_norm_train = scaler_norm_x.transform(X_train)
Y_norm_train = scaler_norm_y.transform(y_train)


# ################qt

scaler_qt_x =  QuantileTransformer(output_distribution='normal').fit(X_train)
scaler_qt_y =  QuantileTransformer(output_distribution='normal').fit(y_train)


X_qt_train = scaler_qt_x.transform(X_train)
Y_qt_train = scaler_qt_y.transform(y_train)


##robust
print(X_train.shape)
print(y_train.shape)
X_train_t = X_train.transpose()
y_train_t = y_train.transpose()
print(X_train.shape,"after")
print(y_train.shape,"after")
scaler_rob_x = RobustScaler().fit(X_train_t)
scaler_rob_y = RobustScaler().fit(y_train_t)


X_rob_train = scaler_rob_x.transform(X_train_t)
Y_rob_train = scaler_rob_y.transform(y_train_t)

X_rob_train = X_rob_train.transpose()
Y_rob_train = Y_rob_train.transpose()

print(X_rob_train.shape)
print(Y_rob_train.shape)
# print(Y_rob_train)

##robustscale
# print(X_train.shape)
# scaler_robsc_x= robust_scale(X_train, axis=1, with_centering=True, with_scaling=True, quantile_range=(25.0, 75.0), copy=True)
# scaler_robsc_y= robust_scale(y_train, axis=1, with_centering=True, with_scaling=True, quantile_range=(25.0, 75.0), copy=True)



# X_robsc_train = scaler_robsc_x
# Y_robsc_train = scaler_robsc_y
# print(X_robsc_train.shape)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:2239: UserWarning: n_quantiles (1000) is greater than the total number of samples (338). n_quantiles is set to n_samples.
  % (self.n_quantiles, n_samples))
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:2239: UserWarning: n_quantiles (1000) is greater than the total number of samples (338). n_quantiles is set to n_samples.
  % (self.n_quantiles, n_samples))


(338, 10000)
(338, 2)
(338, 10000) after
(338, 2) after
(338, 10000)
(338, 2)


In [0]:
# from sklearn.decomposition import PCA
# # # pca - keep 90% of variance


# # print(X_norm)

# # print(Y_norm)
# pca = PCA(n_components = 30)
# pca_fit =  pca.fit(X_rob_train)
# X_new = pca_fit.transform(X_rob_train)
# print(X_new.shape)
# # principal_df = pd.DataFrame(data = principal_components)
# # print(principal_df.head())

# var_exp = pca.explained_variance_ratio_
# print(var_exp.shape)

In [22]:

from sklearn.decomposition import FactorAnalysis

transformer = FactorAnalysis(n_components=30, random_state=0)
factor_fit = transformer.fit(X_rob_train)
X_new = factor_fit.transform(X_rob_train)
X_new.shape

(338, 30)

In [0]:
# from sklearn.decomposition import FastICA

# transformer = FastICA(n_components=30, random_state=0)
# ica_fit = transformer.fit(X_stan_train)
# X_new = ica_fit.transform(X_stan_train)
# X_new.shape


In [0]:

# from sklearn.manifold import Isomap

# transformer = Isomap(n_components=50)

# iso_fit = transformer.fit(X_stan_train)
# X_new = iso_fit.transform(X_stan_train)
# X_new.shape


In [0]:


def baseline_model_30(optimizer='adam'):
    # create model
    model = Sequential()
    model.add(Dense(28, activation='relu', 
                    kernel_initializer = 'he_normal', 
                    input_shape=(30,)))
    model.add(BatchNormalization())
#     model.add(Dropout(0.5))
#     model.add(Dense(30, activation='relu',
#                     kernel_initializer = 'he_normal'))
#       model.add(BatchNormalization())
#     model.add(Dropout(0.5))
    model.add(Dense(12, activation='relu',
                    kernel_initializer = 'he_normal'))
    model.add(BatchNormalization())
#     model.add(Dropout(0.5))
#     model.add(Dense(9, activation='relu',
#                     kernel_initializer = 'he_normal'))
#     model.add(BatchNormalization())
#     model.add(Dropout(0.5))
    model.add(Dense(2, activation='linear', 
                    kernel_initializer='he_normal'))
    model.compile(loss = 'mse', optimizer=optimizer, metrics=['mae'])
#     model.summary()
    return model








# model = Sequential()
# model.add(Dense(30, input_dim=30, kernel_initializer='normal', activation='relu'))
  
# model.add(Dense(20, kernel_initializer='normal', activation='relu'))
# model.add(Dense(12, kernel_initializer='normal', activation='relu'))
# model.add(Dense(4, kernel_initializer='normal'))
# 	# Compile model
# model.summary()
# model.compile(loss='mse', optimizer='adam', metrics=['mse', 'acc'])

In [0]:

def train_data_nn(X_train, y_train):
    
    np.random.seed(42)
    # create model
    estimator = KerasRegressor(build_fn=baseline_model_30, epochs=200, batch_size=5, verbose=0)
    kfold = KFold(n_splits=5, random_state=None )
    results = cross_val_score(estimator, X_train, y_train, cv=kfold)  
    print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))
    return estimator

In [25]:
model = baseline_model_30()

estimator = train_data_nn(X_new, Y_rob_train)

print(X_new.shape)
print(y_train.shape)
history = estimator.fit(X_new,  Y_rob_train, epochs=200, batch_size=5,  verbose=1, validation_split=0.0)

W0711 09:28:14.149700 140493437335424 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0711 09:28:14.190567 140493437335424 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0711 09:28:14.199285 140493437335424 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0711 09:28:14.328209 140493437335424 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0711 09:28

Standardized: -0.10 (0.05) MSE
(338, 30)
(338, 2)
Epoch 1/200
338/338 [==============================] - 2s 5ms/step - loss: 1.9848 - mean_absolute_error: 1.1075
Epoch 2/200
338/338 [==============================] - 1s 2ms/step - loss: 1.2676 - mean_absolute_error: 0.8890
Epoch 3/200
338/338 [==============================] - 1s 2ms/step - loss: 0.8535 - mean_absolute_error: 0.7330
Epoch 4/200
338/338 [==============================] - 1s 2ms/step - loss: 0.5814 - mean_absolute_error: 0.6171
Epoch 5/200
338/338 [==============================] - 1s 2ms/step - loss: 0.4680 - mean_absolute_error: 0.5426
Epoch 6/200
338/338 [==============================] - 1s 2ms/step - loss: 0.3746 - mean_absolute_error: 0.4721
Epoch 7/200
338/338 [==============================] - 1s 2ms/step - loss: 0.2821 - mean_absolute_error: 0.3972
Epoch 8/200
338/338 [==============================] - 1s 2ms/step - loss: 0.2662 - mean_absolute_error: 0.3767
Epoch 9/200
338/338 [==============================] -

In [0]:
def visualize_learning_curve(history):
    # list all data in history
    print(history.history.keys())
    # summarize history for accuracy
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['mean_absolute_error'])
    plt.plot(history.history['val_mean_absolute_error'])
    plt.title('model mean_absolute_error')
    plt.ylabel('mean_absolute_error')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

In [27]:
#row wise'

print(X_test.shape)
X_test_t = X_test.transpose()
print(X_test_t.shape)
scaler_rob_x = RobustScaler().fit(X_test_t)
X_new_test_t = scaler_rob_x.transform(X_test_t)
X_new_test = X_new_test_t.transpose()
print(X_new_test.shape)

y_test_t = y_test.transpose()
scaler_rob_y = RobustScaler().fit(y_test_t)
Y_new_test_t = scaler_rob_y.transform(y_test_t)
Y_new_test = Y_new_test_t.transpose()

X_new_test = factor_fit.transform(X_new_test)
print(X_new_test.shape)





(85, 10000)
(10000, 85)
(85, 10000)
(85, 30)


In [0]:
# # print(X_test.shape)
# # X_test_t = X_test.transpose()
# # print(X_test_t.shape)
# # scaler_rob_x = RobustScaler().fit(X_test)
# X_new_test = scaler_stan_x.transform(X_test)
# # X_new_test = X_new_test_t.transpose()
# print(X_new_test.shape)

# # y_test_t = y_test.transpose()
# # scaler_rob_y = RobustScaler().fit(y_test_t)
# Y_new_test = scaler_stan_y.transform(y_test)
# # Y_new_test = Y_new_test_t.transpose()

# X_new_test = pca_fit.transform(X_new_test)
# print(X_new_test.shape)

# results = model.evaluate(X_new_test, Y_new_test)
# print('loss: ', results[0])
# print('mse: ', results[1])

# print('accuracy',results[2])

# visualize_learning_curve(history)

In [29]:
from sklearn.metrics import mean_squared_error,mean_absolute_error
import math
from sklearn.metrics import max_error

pred = estimator.predict((X_new_test))
print(pred.shape)
# print(pred[:,1])
# print(Y_new_test[:,1])
mse = (mean_squared_error(Y_new_test,pred))
# print(Y_new_test[80])
# print("L")
# print(pred[80])
print(mse)
# Xnew = scaler_norm_x.inverse_transform(Y_new_test[:,3])
# print(rmse)
# ynew = scaler_stan_y.inverse_transform(pred) 

#normalisation
# mul = math.sqrt(np.sum(y_test[58]**2))
# print("L")
# print(y_test[58])
# print(Y_new_test[58]*mul)
# print(pred[58]*mul)


# print(y_test[5])
# print(X_test[5])
# X  = factor_fit.transform(scaler_stan_x.transform(X_test[5].reshape(1, -1)))

# print(scaler_stan_y.transform(y_test[50].reshape(1, -1)))
# print(scaler_stan_y.inverse_transform(Y_new_test[50]))
# print(scaler_stan_y.inverse_transform(estimator.predict(X)))



# Xnew = scaler_stan_x.inverse_transform(Xnew)

# from sklearn.metrics import explained_variance_score
# y_true = Y_new_test
# y_pred= pred
# v = explained_variance_score(y_true, y_pred, multioutput='uniform_average') 
# print(v)
# print(max_error(y_true, y_pred))

# test_df = pd.DataFrame({'y_pred': pred})    
# return test_df

(85, 2)
0.10550674623771744


In [50]:
#row wise


for i in range(len(y_test)):
  
  print(y_test[i],"ytest[i]")
  
  
#   print(X_test[i])
  X_test_t= X_test[i].transpose()
#   print(X_test_t.reshape(-1, 1).shape,"X_test fitted ")
#   print(X_test_t.shape)
  scaler_rob_x = RobustScaler().fit((X_test_t.reshape(-1, 1)))
#   print(X_test_t.reshape(-1, 1))                            
  X = (scaler_rob_x.transform(X_test_t.reshape(-1, 1)))
#   print(X.shape,"X shape")
  I = factor_fit.transform(X.transpose())
#   print(I.shape,"I")
  pred = estimator.predict(I)
#   print(pred.shape,"pred.shape")
  
  y_test_ti = y_test[i].transpose()
#   print(y_test_ti.reshape(-1, 1).shape,"ytest tranposed shape")
  scaler_rob_y = RobustScaler().fit(y_test_ti.reshape(-1, 1))
  final_t = scaler_rob_y.inverse_transform(pred.reshape(-1, 1))
                                          
#   print(final_t.shape)
  final = final_t.transpose()
                                          
  print(final[0])
#   Y_new_test_t = scaler_rob_y.transform(y_test_ti)
#   Y_new_test = Y_new_test_t.transpose()

#   X= X.tranpose()
#   # print(scaler_stan_y.transform(y_test[50].reshape(1, -1)))
#   # print(scaler_stan_y.inverse_transform(Y_new_test[50]))
#   pred = (estimator.predict(X))
# #   print(scaler_stan_y.inverse_transform(estimator.predict(X)))
#   mul = math.sqrt(np.sum(y_test[i]**2))
# #   print("L")
# #   print(y_test[58])
# #   print(Y_new_test[58]*mul)
# #   print(pred[58]*mul)
#   pred = pred.tranform()
#   l = scaler_rob_y.inverse_transform(pred)
#   l=l.transpose()
# #   l = (estimator.predict(X))*mul
#   print(l,"l")
  h = abs(final-y_test[i])
#   print(h,"h")
  o=np.divide(h,y_test[i])
  print(o*100,"percentage") 


[ 26.33768194 498.06189793] ytest[i]
[370.9384  152.85501]
[[1308.39420054   69.31003724]] percentage
[130.     564.2657] ytest[i]
[114.80985 567.00903]
[[11.68472877  0.48617756]] percentage
[143.2    401.3125] ytest[i]
[155.5722  379.36905]
[[8.63980767 5.46792112]] percentage
[140.     578.4063] ytest[i]
[137.20892 574.4504 ]
[[1.99362619 0.68393473]] percentage
[125.     610.0782] ytest[i]
[108.43781 612.477  ]
[[13.24974976  0.39319381]] percentage
[-1.3    -2.0156] ytest[i]
[-2.0829372 -1.2430351]
[[-60.22594158 -38.32927773]] percentage
[149.5    401.3125] ytest[i]
[201.40106 341.68857]
[[34.71642944 14.85723268]] percentage
[126.9    669.1407] ytest[i]
[140.89185 645.00244]
[[11.02588314  3.60735173]] percentage
[140.     405.9922] ytest[i]
[313.73395 231.22076]
[[124.09567697  43.04797871]] percentage
[170.     612.8125] ytest[i]
[166.4347  606.08563]
[[2.09723977 1.09770406]] percentage
[140.     401.3125] ytest[i]
[158.67877 377.6724 ]
[[13.34197998  5.8906977 ]] percentage


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: RuntimeWarning: divide by zero encountered in true_divide


In [63]:
####creating new X,Y
X1 = input_1_arr[:,0:10000]
Y1 = input_1_arr[:,10000:10004]

X=X1
Y=Y1[:,2:4]
print(Y)
Y_new = np.zeros((Y.shape[0],2))
for i in range(len(Y)):
  
  print(Y[i],"y")
#   print(Y.shape)
  
  

  X_t= X[i].transpose()
#   print(X_test_t.reshape(-1, 1).shape,"X_test fitted ")
#   print(X_test_t.shape)
  scaler_rob_x = RobustScaler().fit((X_t.reshape(-1, 1)))
#   print(X_test_t.reshape(-1, 1))                            
  Xi = (scaler_rob_x.transform(X_t.reshape(-1, 1)))
#   print(X.shape,"X shape")
  I = factor_fit.transform(Xi.transpose())
#   print(I.shape,"I")
  pred = estimator.predict(I)
#   print(pred.shape,"pred.shape")
  
  Y_ti =Y[i].transpose()
#   print(y_test_ti.reshape(-1, 1).shape,"ytest tranposed shape")
  scaler_rob_y = RobustScaler().fit(Y_ti.reshape(-1, 1))
  final_t = scaler_rob_y.inverse_transform(pred.reshape(-1, 1))
                                          
#   print(final_t.shape)
  final = final_t.transpose()
                                          
  print(final[0])

  h = abs(final-Y[i])
#   print(h,"h")
  o=np.divide(h,Y[i])
  print(o*100,"percentage") 
  
  Y_new[i]=final[0]
#   print(Y[i],"Y")
  print(Y_new[i])

# print(Y_new[:,0])




[[ 2.57760406e+02  2.91996063e+02]
 [ 2.67804047e+02  2.69610657e+02]
 [ 2.85054901e+02  2.85258087e+02]
 [ 2.70117096e+02  2.91277222e+02]
 [ 2.66462524e+02  2.89624847e+02]
 [ 2.82232574e+02  2.82232544e+02]
 [ 2.64577836e-01  2.68248320e-01]
 [ 2.60365397e-01  2.65748560e-01]
 [ 1.12380302e+02  6.27531982e+02]
 [ 1.78185768e+01  6.75317993e+02]
 [ 1.30478317e+02  5.91765747e+02]
 [ 1.44108017e+02  5.99537537e+02]
 [ 6.85454102e+01  6.42164795e+02]
 [ 3.74985275e+01  6.52746033e+02]
 [ 1.21111832e+02  5.65515869e+02]
 [ 1.18575783e+02  5.60275513e+02]
 [ 1.89996674e+02  5.25277344e+02]
 [ 9.38728104e+01  5.97802490e+02]
 [ 2.69383972e+02  2.85293762e+02]
 [ 2.94082916e+02  2.88676117e+02]
 [ 2.78878204e+02  2.86848755e+02]
 [ 2.84867676e+02  2.84914154e+02]
 [ 2.73436981e+02  2.88515808e+02]
 [ 2.63540497e+02  2.60155243e+02]
 [ 2.86197205e+02  2.86138428e+02]
 [-1.57966766e+02  1.38421967e+02]
 [-1.53227020e+02  1.54151367e+02]
 [ 9.65055161e+01  5.56644775e+02]
 [ 8.42297134e+01  5

In [78]:
X1 = X1
Y1 = Y1
print(Y1)

# print(Y)

# print(Y_new[:,0])

Y1[:,2]= Y_new[:,0]
Y1[:,3]= Y_new[:,1]
print(Y1)

[[  1.          89.99       263.93359375 285.41305542]
 [  1.          89.95       268.42147827 268.95645142]
 [  1.          36.97       285.13900757 285.17358398]
 ...
 [  1.          36.93       271.62432861 275.29064941]
 [  1.          36.9        264.49822998 285.29458618]
 [  1.          36.75       278.24633789 278.3482666 ]]
[[  1.          89.99       263.93359375 285.41305542]
 [  1.          89.95       268.42147827 268.95645142]
 [  1.          36.97       285.13900757 285.17358398]
 ...
 [  1.          36.93       271.62432861 275.29064941]
 [  1.          36.9        264.49822998 285.29458618]
 [  1.          36.75       278.24633789 278.3482666 ]]


In [79]:
X1_new = np.concatenate((X1,Y1[:,2:4]),axis=1)
print(X1_new.shape)
Y1_new = Y1[:,0:2]
# print(Y1_new)

(423, 10002)


In [0]:
#train and test

X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X1_new, Y1_new, test_size=0.20)


In [81]:
#standardise


# ######minmax
scaler_min_x = MinMaxScaler().fit(X_train_c)
scaler_min_y = MinMaxScaler().fit(y_train_c)

X_minmax_train = scaler_min_x.transform(X_train_c)
Y_minmax_train = scaler_min_y.transform(y_train_c)


# print(X)
# print(Y)
#####standard

scaler_stan_x = StandardScaler().fit(X_train_c)
scaler_stan_y = StandardScaler().fit(y_train_c)


X_stan_train = scaler_stan_x.transform(X_train_c)
Y_stan_train = scaler_stan_y.transform(y_train_c)

#######normlised
scaler_norm_x = Normalizer().fit(X_train_c)
scaler_norm_y = Normalizer().fit(y_train_c)


X_norm_train = scaler_norm_x.transform(X_train_c)
Y_norm_train = scaler_norm_y.transform(y_train_c)


# ################qt

scaler_qt_x =  QuantileTransformer(output_distribution='normal').fit(X_train_c)
scaler_qt_y =  QuantileTransformer(output_distribution='normal').fit(y_train_c)


X_qt_train = scaler_qt_x.transform(X_train_c)
Y_qt_train = scaler_qt_y.transform(y_train_c)


##robust
# print(X_train.shape)
# print(y_train.shape)
# X_train_t = X_train.transpose()
# y_train_t = y_train.transpose()
# print(X_train.shape,"after")
# print(y_train.shape,"after")
scaler_rob_x = RobustScaler().fit(X_train_c)
scaler_rob_y = RobustScaler().fit(y_train_c)


X_rob_train = scaler_rob_x.transform(X_train_c)
Y_rob_train = scaler_rob_y.transform(y_train_c)

# X_rob_train = X_rob_train.transpose()
# Y_rob_train = Y_rob_train.transpose()

# print(X_rob_train.shape)
# print(Y_rob_train.shape)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:2239: UserWarning: n_quantiles (1000) is greater than the total number of samples (338). n_quantiles is set to n_samples.
  % (self.n_quantiles, n_samples))
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:2239: UserWarning: n_quantiles (1000) is greater than the total number of samples (338). n_quantiles is set to n_samples.
  % (self.n_quantiles, n_samples))


In [0]:
#apply PCA on X1_new
transformer = FactorAnalysis(n_components=30, random_state=0)
factor_fit = transformer.fit(X_stan_train[:,0:10000])
X_new1 = factor_fit.transform(X_stan_train[:,0:10000])
print(X_new1.shape)
X_new1 = np.concatenate((X_new1,X_rob_train[:,10000:10002]),axis=1)
print(X_new1.shape)
# print((X_rob_train[:,0:10000].shape))

In [0]:


def baseline_model_31(optimizer='adam'):
    # create model
    model = Sequential()
    model.add(Dense(28, activation='relu', 
                    kernel_initializer = 'he_normal', 
                    input_shape=(32,)))
    model.add(BatchNormalization())
#     model.add(Dropout(0.5))
#     model.add(Dense(30, activation='relu',
#                     kernel_initializer = 'he_normal'))
#       model.add(BatchNormalization())
#     model.add(Dropout(0.5))
    model.add(Dense(12, activation='relu',
                    kernel_initializer = 'he_normal'))
    model.add(BatchNormalization())
#     model.add(Dropout(0.5))
#     model.add(Dense(9, activation='relu',
#                     kernel_initializer = 'he_normal'))
#     model.add(BatchNormalization())
#     model.add(Dropout(0.5))
    model.add(Dense(2, activation='linear', 
                    kernel_initializer='he_normal'))
    model.compile(loss = 'mse', optimizer=optimizer, metrics=['mae'])
#     model.summary()
    return model



In [0]:
def train_data_nn_1(X_train, y_train):
    
    np.random.seed(42)
    # create model
    estimator = KerasRegressor(build_fn=baseline_model_31, epochs=200, batch_size=5, verbose=0)
    kfold = KFold(n_splits=5, random_state=None )
    results = cross_val_score(estimator, X_train, y_train, cv=kfold)  
    print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))
    return estimator

In [71]:
model1 = baseline_model_31()

estimator1 = train_data_nn_1(X_new1, Y_rob_train)

# print(X_new.shape)
# print(y_train.shape)
history = estimator1.fit(X_new1,  Y_rob_train, epochs=200, batch_size=5,  verbose=1, validation_split=0.0)

Standardized: -4.45 (2.48) MSE
Epoch 1/200
338/338 [==============================] - 3s 9ms/step - loss: 4.6665 - mean_absolute_error: 1.4277
Epoch 2/200
338/338 [==============================] - 1s 2ms/step - loss: 4.3824 - mean_absolute_error: 1.2925
Epoch 3/200
338/338 [==============================] - 1s 2ms/step - loss: 3.8833 - mean_absolute_error: 1.1648
Epoch 4/200
338/338 [==============================] - 1s 2ms/step - loss: 4.0274 - mean_absolute_error: 1.1474
Epoch 5/200
338/338 [==============================] - 1s 2ms/step - loss: 3.7320 - mean_absolute_error: 1.0798
Epoch 6/200
338/338 [==============================] - 1s 2ms/step - loss: 3.5354 - mean_absolute_error: 1.0259
Epoch 7/200
338/338 [==============================] - 1s 2ms/step - loss: 3.5431 - mean_absolute_error: 1.0118
Epoch 8/200
338/338 [==============================] - 1s 2ms/step - loss: 3.7488 - mean_absolute_error: 1.0178
Epoch 9/200
338/338 [==============================] - 1s 2ms/step - loss

In [73]:
#row wise'


print(X_test_c.shape)

X_rob_test_c = scaler_rob_x.transform(X_test_c)

print(X_rob_test_c.shape)

Y_rob_test_c = scaler_rob_y.transform(y_test_c)


X_new_test_c = factor_fit.transform(X_rob_test_c[:,0:10000])
print(X_new_test.shape)


X_new_test_c = np.concatenate((X_new_test_c,X_rob_test_c[:,10000:10002]),axis=1)
print(X_new_test_c.shape)



(85, 10002)
(85, 10002)
(85, 30)
(85, 32)


In [74]:
from sklearn.metrics import mean_squared_error,mean_absolute_error
import math
from sklearn.metrics import max_error

pred_c = estimator1.predict((X_new_test_c))
print(pred_c.shape)

mse = (mean_squared_error(Y_rob_test_c,pred_c))

print(mse)

(85, 2)
3.601147779412792


In [77]:
for i in range(len(y_test_c)):
  
  print(y_test_c[i],"ytest[i]")
#   print(X_test_c[i].shape)
  X_c = (scaler_rob_x.transform(X_test_c[i].reshape(1, -1)))

  I = factor_fit.transform(X_c[:,0:10000])
  I = np.concatenate((I,X_c[:,10000:10002]),axis=1)
#   print(I.shape,"I shape")

  pred_c = estimator1.predict(I)
#   print(pred_c.shape,"pred_c.shape")
  
 

  
  final = scaler_rob_y.inverse_transform(pred_c.reshape(1, -1))
                                          

                                          
  print(final[0],"final")

  h = abs(final[0]-y_test_c[i])
#   print(h,"h")
  o=np.divide(h,y_test_c[i])
  print(o*100,"percentage") 


[ 1.   52.27] ytest[i]
[ 0.9997615 57.01222  ] final
[0.02384782 9.07254348] percentage
[ 1.   49.09] ytest[i]
[ 1.000071 47.91555 ] final
[0.00710487 2.39244198] percentage
[ 1.   51.35] ytest[i]
[ 0.99988544 58.224445  ] final
[1.14560127e-02 1.33874301e+01] percentage
[ 1.   36.85] ytest[i]
[ 0.9999452 42.667305 ] final
[5.47766685e-03 1.57864450e+01] percentage
[ 1.  52.4] ytest[i]
[ 0.9997674 60.80906  ] final
[ 0.02325773 16.04782279] percentage
[ 1.   87.09] ytest[i]
[ 0.99991244 62.887135  ] final
[8.75592232e-03 2.77906366e+01] percentage
[ 1.   50.75] ytest[i]
[ 1.00004  49.829933] final
[0.00400543 1.81293957] percentage
[ 1.   51.05] ytest[i]
[ 0.9999961 53.34677  ] final
[3.87430191e-04 4.49906217e+00] percentage
[ 1.   44.44] ytest[i]
[ 1.0000199 46.192963 ] final
[1.99079514e-03 3.94456041e+00] percentage
[ 1.   56.39] ytest[i]
[ 0.9999309 51.849094 ] final
[6.90817833e-03 8.05267886e+00] percentage
[ 1.         50.40596493] ytest[i]
[ 1.0000563 59.480927 ] final
[5.6266

In [0]:
# for i in range(len(y_test)):
#   print(y_test[i],"ytest[i]")
#   X=X_test[i]
# #   print(X.reshape(1, -1).shape)
#   pred = estimator.predict(pca_fit.transform(X.reshape(1, -1)))
#   print(pred)
#   print("-------")

In [0]:
# from sklearn.metrics import r2_score
  
# r2_score(Y_new_test[:,:], pred[:,:], multioutput='variance_weighted') 

In [0]:


# for i in range(len(y_test)):
  
#   print(y_test[i],"ytest[i]")
  
  
# #   print(X_test[i])
# #   X_test[i]= X_test[i].transpose()
# #   print(X_test[i].reshape(-1, 1).shape,"X_test fitted ")
# #   scaler_rob_x = RobustScaler().fit((X_test[i].reshape(-1, 1)))
# #   print(X_test[i].reshape(-1, 1))                            
# #   X = (scaler_stan_x.transform(X_test[i].reshape(-1, 1)))
  
# #   pred = estimator.predict(X)
# #   print(pred.shape,"pred.shape")
#   print()
# #   y_test_ti = y_test[i].transpose()
# #   print(y_test_ti.reshape(-1, 1).shape,"ytest tranposed shape")
# #   scaler_rob_y = RobustScaler().fit(y_test_ti.reshape(-1, 1))
#   final = scaler_stan_y.inverse_transform(pred[i].reshape(-1, 1))
                                          
# #   print(final_t.shape)
# #   final = final_t.transpose()
                                          
#   print(final)
# #   Y_new_test_t = scaler_rob_y.transform(y_test_ti)
# #   Y_new_test = Y_new_test_t.transpose()

# #   X= X.tranpose()
# #   # print(scaler_stan_y.transform(y_test[50].reshape(1, -1)))
# #   # print(scaler_stan_y.inverse_transform(Y_new_test[50]))
# #   pred = (estimator.predict(X))
# # #   print(scaler_stan_y.inverse_transform(estimator.predict(X)))
# #   mul = math.sqrt(np.sum(y_test[i]**2))
# # #   print("L")
# # #   print(y_test[58])
# # #   print(Y_new_test[58]*mul)
# # #   print(pred[58]*mul)
# #   pred = pred.tranform()
# #   l = scaler_rob_y.inverse_transform(pred)
# #   l=l.transpose()
# # #   l = (estimator.predict(X))*mul
# #   print(l,"l")
# #   h = abs(l-y_test[i])
# # #   print(h,"h")
# #   o=np.divide(h,y_test[i])
# #   print(o*100,"percentage")

In [46]:
 fstanprint(X[80])
print(Y[91])
X_new_test = scaler_stan_x.transform(X[91].reshape(1, -1))

Y_new_test = scaler_stan_y.transform(Y[91].reshape(1, -1))

X_new_test = ica_fit.transform(X_new_test)
pred = estimator.predict(X_new_test)
ynew = scaler_stan_y.inverse_transform(pred) 
print(ynew)

NameError: ignored

In [0]:
print(history.history.keys())
# "Loss"
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')

plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [0]:
# "mean_squared_error"
plt.plot(history.history['mean_squared_error'])
plt.plot(history.history['val_mean_squared_error'])
plt.title('model mean_squared_error')
plt.ylabel('mean_squared_error')

plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [0]:
 # "acc"
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])

plt.title('model acc')
plt.ylabel('acc')

plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')

plt.show()